In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import os

# Measuring the "time of arrival" of a pulsar
## Crab Pulsar Experiment Part 3.2

This notebook can be used for the first part of the crab timing part of the experiment. Here you will find code that reads your data files (pulsar archives) and matches the observed pulse against a _standard template_ of the crab pulsar. This standard template is a noise-free model of the shape of the pulse and we can measure the time that the pulse arrived to very high precision by correlating the known shape of the pulse (the template) with the noisy data.

The code is in four main parts:
 1. Specifying the data location
 2. Dedispersion of the data ( **insert your code from Part 1 here** )
 3. Measuring the time of arrival (ToA)
 4. Visualising the result
 
_You will need to run all three parts in sequence to get the right results._ Visualising the output is important to be sure you have got sensible results.



# Load Data

In [ ]:
# Connect (mount) your google drive as a virtual directory accessible by this python code.
from google.colab import drive         # Import the python module that allows you to access your google drive
rootpathdrive = '/content/drive'       # This will be the directory as which your google drive will be known
drive.mount(rootpathdrive)             # Now connect to this google drive. 

# At first use it will ask you to click on a link, after which you should give permission 
# for outside processes to access your google drive. A authorization code is generated which should be entered 
#(this is explained in https://colab.research.google.com/notebooks/io.ipynb#scrollTo=XDg9OBaYqRMd).

In [ ]:
# A good test to do is if you can see the contents of the directory in which you work on your google drive.
# Here "My Drive" refers to the "root" of your google drive.
# By default your notebook should be in a directory called Colab Notebooks.
# This template assumes all files you want to read in are copied in the
# same directory. Note the slash at the end of the first line.
pathcrabtemplate = rootpathdrive+'/'+'My Drive/Crab_pulsar_template/'
filelist = []
for (dirpath, dirnames, filenames) in os.walk(pathcrabtemplate):
    filelist.extend(filenames)
    break
print (filelist)   # Show the contents of your working directory. At least your notebook should show up here.

In [ ]:
filename=os.path.join(pathcrabtemplate,"mydata/20220907_233401_B0531+21.npz")# Enter the filename of your data here
obsdata = np.load(filename)
print(obsdata['header'])
data=obsdata['data']

print("'Guess Period':",obsdata['approx_period'])

In [ ]:
templatefilename=os.path.join(pathcrabtemplate,"template.txt")# Enter the filename of your data here
template=np.loadtxt(templatefilename)
plt.plot(template)
plt.show()

In [ ]:
##
#  This function will shift each row of a 2-d (3-d) array by the the number of columns specified in the "shifts" array.
#  data_in - the 2-d (3-d) array to work on
#  shifts - the shifts to apply
#  Returns: The shifted array
##
def shift_rows(data_in, shifts):
    shifted = np.zeros_like(data_in)
    if data_in.ndim == 3:
        for i in range(data_in.shape[0]):
            shifted[i] = shift_rows(data_in[i],shifts)
    else:
        for i in range(data_in.shape[0]):
            shifted[i] = np.roll(data_in[i],int(shifts[i]))
    return shifted


## Computing the time of arrivals
**Please don't worry too much about how this code works.** This is based on research software used for experiments ranging searching for planets around pulsars to measuring gravitational waves passing over the earth. It has been annotated with a few comments for the curious, but you can mostly just treat this as a black box you can use to get time of arrival measurements for your pulsar data.
If you are desparate to dig into the details, this code is based on the ToA estimation procedure described in Appendix A of Taylor et al. 1992. (https://www.jstor.org/stable/53915)

In [ ]:

def get_toas(ddfreq_averaged,obsdata,plots=True):
    
    times=obsdata['times'] # The time of phase zero for each subint
    approx_period=obsdata['approx_period'] # The approximate period of the pulsar
    toas=[]
    toa_errs=[]
    tempo2_toas=[]

    # Equation A7 in Taylor 1992
    def get_dchi(tau,N,nbin):
        dphi = np.angle(xspec)[1:N]

        k=np.arange(1,N)

        dchi = np.sum(k*np.abs(f_prof[1:N])*np.abs(f_template[1:N]) * np.sin(dphi + 2*np.pi*k*tau/nbin))
        return dchi


    # Equation A9 in Taylor 1992
    def get_b(tau,N,nbin):
        dphi = np.angle(xspec)[1:N]
        k=np.arange(1,N)
        scale = np.sum(np.abs(f_prof[1:N])*np.abs(f_template[1:N]) * np.cos(dphi + 2*np.pi*k*tau/nbin))
        scale /= np.sum(np.abs(f_template[1:N])**2)
        return scale


    # Equation A10 in Taylor 1992
    def get_sigma_tau(tau,N,nbin,b):
        dphi = np.angle(xspec)[1:N]
        k=np.arange(1,N)
        chi2=np.sum(np.abs(f_prof[1:N])**2 + b**2*np.abs(f_template[1:N]) )-2*b*np.sum(np.abs(f_prof[1:N])*np.abs(f_template[1:N]) * np.cos(dphi + 2*np.pi*k*tau/nbin))
        sigma2 = chi2/(N-1)
        de = np.sum((k**2) * np.abs(f_prof[1:N])*np.abs(f_template[1:N]) * np.cos(dphi + 2*np.pi*k*tau/nbin))
        fac=nbin/(2*np.pi)
        return np.sqrt(sigma2/(2*b*de))*fac

    # Just for plotting, rotates an array by a fractional phase shift using Fourier transform
    def rotate_phs(ff,phase_shift):
        fr = ff*np.exp(1.0j*2*np.pi*np.arange(len(ff))*phase_shift)
        return np.fft.irfft(fr)

    
    # Loop over every sub integration
    for ip in range(len(ddfreq_averaged)):
        prof=ddfreq_averaged[ip]
        nbin=len(prof)

        # We are going to do a cross correlation by means of the Fourier transform and the Wiener-Kinchin theorem
        f_template = np.fft.rfft(template)
        f_prof = np.fft.rfft(prof)

        # The cross correlation of a and b is the inverse transform of FT(a) times the conjugate of FT(b)
        xspec= f_template * f_prof.conj() # "cross spectrum"
        xcor = np.fft.irfft(xspec) # Cross correlation

        ishift = np.argmax(np.abs(xcor)) # estimate of the shift directly from the peak cross-correlation

        # We need to define some bounds to search. (Actually this might not be optimal)
        lo=ishift-1
        hi=ishift+1
        nh=len(xspec)
        # We minimise the chisquare parameter by findng the root of it's derivatiive following Taylor 1992 
        # This root_scalar method uses the 'Brent 1973' algorithm for root finding.
        ret = opt.root_scalar(get_dchi,bracket=(lo,hi),x0=ishift,args=(nh,nbin),method='brentq')

        # tau is the bin shift between data and template, which will become our ToA
        tau=ret.root
        # Again folow the math of Taylor 1992 to get the scale factor, which it calls 'b'
        scale=get_b(tau,nh,nbin)
        # And finally given the shift and scale we can find the uncertainty on the shift.
        sigma_tau=get_sigma_tau(tau,nh,nbin,scale)

        # Phase shift is bin shift divided by nbins
        phase_shift = tau/nbin

        # ToA is the phase shift converted to a time shift and added to the time of phase zero.
        toa = times[ip]+approx_period*tau/nbin/86400.0
        toa_err = approx_period*sigma_tau/nbin
        tempo2_toa=" test 611 {:.16f} {} jb42\n".format(toa,toa_err)

        toas.append(toa)
        toa_errs.append(toa_err)
        tempo2_toas.append(tempo2_toa)

        phase=np.linspace(0,1,nbin)

        rotate_and_scaled_template=scale*rotate_phs(f_template,phase_shift)
        diff=prof-rotate_and_scaled_template

        d= np.amax(prof)-np.amin(prof)
        if plots:
            # And do some plotting...
            plt.figure(figsize=(12,8))
            plt.xlabel("Pulse Phase")
            plt.ylabel("Flux Density (arbitrary)")

            plt.title(r"{} Profile {:d}:  $\delta t =$ {:.3f} $\pm$ {:.3f} ms".format(obsdata['source_name'],ip,1e3*approx_period*phase_shift,1e3*toa_err))
            plt.step(phase,prof,color='black',linewidth=1.0,label="Data")
            plt.plot(phase,rotate_and_scaled_template,color='red',label="Template")

            plt.step(phase,diff-d,color='green',linewidth=1.0,alpha=0.8,label=r"Data$-$template")
            plt.axhline(-d,ls=":",color='k')

            plt.xlim(0,1)
            plt.ylim(np.amin(prof)-1.2*d,np.amax(prof)+0.1*d)

            plt.legend(loc="lower left",ncol=3)

            plt.show()
            plt.close()
    return np.array(toas),np.array(toa_errs)


# Your code - Dedisperse and make the ToAs
Here you can insert your dedispersion code from part 1, and then call the function to make toas

In [ ]:
## Insert your de-dispersion code here
# ... 
# Create a de-dispersed frequency-averaged data called ddfreq_averaged
# ddfreq_averaged = 
#
nsub, nchan,nphase = data.shape
ichan=np.arange(nchan)

toas,toa_errs = get_toas(ddfreq_averaged, obsdata)

with open(filename+".toas.txt","w") as outf:
    for t,e in zip(toas,toa_errs):
        print("{:.18f} {:.3g}".format(t,e))
        outf.write("{:.18f} {:.3g}\n".format(t,e))